## YOLO ALGORITHM - You Only Look Once

In [1]:
import os
os.getcwd()

'F:\\Satish\\Learning\\Python\\OpenCV'

In [2]:
os.chdir('F://Satish//Learning//Python//OpenCV//DATA/06-YOLOv3/')

In [3]:
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

Using TensorFlow backend.


# Processing the Input Image

In [4]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

## Grab Classes from COCO Class Text File

In [5]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

## Draw a Rectangle once the image identified, give a class to the image and labelling the percent sure of the image

In [6]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

## Detecting the Image using YOLO Algorithm

In [7]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

## Detecting the Video Frame by Frame using YOLO Algorithm

In [8]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()

## Instance Creation of Yolo

In [9]:
%%time

yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

Wall time: 8.23 s


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Testing on Images

In [10]:
f = 'Satish_Car.jpg'
path = 'images/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 11.53s
class: person, score: 0.98
box coordinate x,y,w,h: [295.36547899 324.49220896 454.95671153 730.01974583]
class: car, score: 0.66
box coordinate x,y,w,h: [ 61.53802872 528.62097502 961.38769627 557.03992367]



True

## Testing on Videos

In [13]:
%%time

# # detect videos one at a time in videos/test folder    .

video = 'Satish_Video.MOV'
detect_video(video, yolo, all_classes)

time: 1.92s
class: person, score: 0.91
box coordinate x,y,w,h: [ 86.11302853 229.18164968 184.35782433 588.17773104]
class: person, score: 0.89
box coordinate x,y,w,h: [746.03096008 263.63494098 209.60237503 502.74299026]
class: person, score: 0.89
box coordinate x,y,w,h: [323.73476028 244.50896144 159.3170929  564.50738668]
class: person, score: 0.88
box coordinate x,y,w,h: [963.92360687 251.15592599 226.29916191 552.09343672]
class: person, score: 0.86
box coordinate x,y,w,h: [1320.89927673  209.91779923  221.8367815   539.36148405]

time: 1.87s
class: person, score: 0.96
box coordinate x,y,w,h: [751.76319122 254.86609697 204.72538948 510.6188035 ]
class: person, score: 0.94
box coordinate x,y,w,h: [962.54802704 230.64095378 220.92790604 595.4263258 ]
class: person, score: 0.92
box coordinate x,y,w,h: [318.4725666  248.78593683 162.72902012 556.60926819]
class: person, score: 0.90
box coordinate x,y,w,h: [ 84.09912586 226.36829138 191.85822487 589.35530663]
class: person, score: 0.84

## THANK YOU